In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
%cd /kaggle/working
!git clone https://github.com/valkryhx/chatGLM-6B-QLoRA
%cd chatGLM-6B-QLoRA
!pip install -r requirements.txt

/kaggle/working
fatal: destination path 'chatGLM-6B-QLoRA' already exists and is not an empty directory.
/kaggle/working/chatGLM-6B-QLoRA


In [3]:
!pip install bitsandbytes==0.41.1
!pip install peft==0.5.0 
!pip install accelerate==0.21.0 
!pip install trl==0.6.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 7.3 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.39.0
    Uninstalling bitsandbytes-0.39.0:
      Successfully uninstalled bitsandbytes-0.39.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: peft
    Found existing installation: peft 0.4.0
    Uninstalling peft-0.4.0:
      Successfully uninstalled peft-0.4.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.9 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.20.3
    Uninstalling accelerate-0.20.3:
      Successfully uninstalled accelerate-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.0/110.0 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: trl
    Found existing installation: trl 0.5.0
    Uninstalling trl-0.5.0:
      Successfully uninstalled trl-0.5.0


 * # <font color=red>首先sft  with qlora </font> 

In [5]:
%cd /kaggle/working/chatGLM-6B-QLoRA
!git pull --all --force 
!deepspeed --include localhost:0,1  train_qlora_deepspeed_zero.py  \
  --train_args_json luzi.json \
  --model_name_or_path THUDM/chatglm2-6b \
  --output_dir output_yungaun_0827_v1 \
  --num_train_samples -1 \
  --num_eval_samples 100 \
  --train_data_path ./data/augment_staff298_qa70and14  \
  --eval_data_path  ./data/augment_staff298_qa70and14    \
  --max_input_length 256 \
  --max_output_length 400 \
  --lora_rank 64 \
  --lora_dropout 0.05 \
  --compute_dtype fp16 \
  --per_device_train_batch_size 2 \
  --per_device_eval_batch_size 2  \
  --gradient_accumulation_steps 1 \
  --learning_rate  1e-5 \
  --num_train_epochs  20  \
  --save_total_limit 2 \
  --load_in_4bit True \
--deepspeed ds_zero2_config.json

/kaggle/working/chatGLM-6B-QLoRA
Fetching origin
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), 709 bytes | 354.00 KiB/s, done.
From https://github.com/valkryhx/chatGLM-6B-QLoRA
   addf61b..623d4a3  main       -> origin/main
Updating addf61b..623d4a3
Fast-forward
 train_qlora_deepspeed_zero.py | 3 ++-
 1 file changed, 2 insertions(+), 1 deletion(-)
[2023-08-29 12:06:28,035] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to l

In [6]:
%cd /kaggle/working
!git clone -b medGPT_0828  https://github.com/valkryhx/MedicalGPT

/kaggle/working
fatal: destination path 'MedicalGPT' already exists and is not an empty directory.


In [7]:
%cd MedicalGPT
!git status

/kaggle/working/MedicalGPT
On branch medGPT_0828
Your branch is up to date with 'origin/medGPT_0828'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   data/vocab/word_freq.txt

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	cache/
	outputs-dpo-v1/
	outputs-dpo-yunguan-v1/
	state.db

no changes added to commit (use "git add" and/or "git commit -a")


In [9]:
!cat ./requirements.txt
# 发现这项目的依赖和上面重合 不用再装一次

loguru
transformers==4.30.2
sentencepiece
datasets==2.12.0
tqdm
tensorboard
tqdm>=4.47.0
peft>=0.5.0
accelerate==0.21.0
trl>=0.6.0
bitsandbytes==0.39.0


# 参考这里 colab的dpo运行demo  是使用base model（或者sft合并后的model 仍然是base model格式）传入的
https://colab.research.google.com/drive/1kMIe3pTec2snQvLBA00Br8ND1_zwy3Gr?usp=sharing

# 使用运管数据 ./data/reward_yunguan
# <font color=red>注意这里没有使用--qlora True 说明是使用普通lora来训练base model的</font>  代码中会加载两次chatglm2 一次作为可训练的model  一次作为ref_model 注意不要oom

# 注意dpo_training.py 代码中 optim默认=adamw_hf  在lora训练时可以 但是qlora训练一定换成paged_adamw_8bit 不然loss=0 其他指标为nan
# 注意dpo_training.py 代码中 find_all_linear_names(peft_model, int4=False, int8=False)定义时加入了判断是否int4 or int8 量化 lora训练一般不量化 此时cls = torch.nn.Linear  注意find_all_linear_names的层<font color=red>好像比我自己写qlora时的find_all_linear_names多了一层default  今天测试好像没多这个奇怪的default层</font>

 # --lora_rank 64 \ --lora_alpha 32 \ 会oom
 # 我改成 --lora_rank 8 \ --lora_alpha 16 \ 没有oom
 # learning_rate 默认 5e-4 太大导致loss不稳定 我换成较小的1e-5
 # warmup_steps 默认100 我改成 10
 # <font color=red>train/eval数据从./data/reward_yunguan 换成./data/reward loss瞬间下降 毕竟这个只有100条数据 而reward_yunguan有2800多条</font>
 # [未实施]optim由adamw_hf 改成 paged_adamw_32bit？ 后者是trl官网dpo例子用的

In [7]:
%cd /kaggle/working/MedicalGPT
!git pull --force --all
!python dpo_training.py \
    --model_type chatglm \
    --model_name_or_path THUDM/chatglm2-6b \
    --train_file_dir ./data/reward \
    --validation_file_dir ./data/reward \
    --learning_rate 1e-5 \
    --warmup_steps 10 \
    --per_device_train_batch_size 1 \
    --per_device_eval_batch_size 1 \
    --do_train \
    --do_eval \
    --use_peft True \
    --max_train_samples 1000 \
    --max_eval_samples 20 \
    --max_steps 100 \
    --eval_steps 10 \
    --save_steps 40 \
    --max_source_length 256 \
    --max_target_length 128 \
    --output_dir outputs-dpo-v1 \
    --target_modules all \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --torch_dtype float16 \
    --fp16 True \
    --device_map auto \
    --report_to tensorboard \
    --remove_unused_columns False \
    --gradient_checkpointing True \
    --cache_dir ./cache

/kaggle/working/MedicalGPT
Fetching origin
Already up to date.
^C
Traceback (most recent call last):
  File "/kaggle/working/MedicalGPT/dpo_training.py", line 16, in <module>
    from peft import LoraConfig, TaskType
  File "/opt/conda/lib/python3.10/site-packages/peft/__init__.py", line 22, in <module>
    from .auto import (
  File "/opt/conda/lib/python3.10/site-packages/peft/auto.py", line 30, in <module>
    from .config import PeftConfig
  File "/opt/conda/lib/python3.10/site-packages/peft/config.py", line 24, in <module>
    from .utils import CONFIG_NAME, PeftType, TaskType
  File "/opt/conda/lib/python3.10/site-packages/peft/utils/__init__.py", line 22, in <module>
    from .other import (
  File "/opt/conda/lib/python3.10/site-packages/peft/utils/other.py", line 21, in <module>
    import accelerate
  File "/opt/conda/lib/python3.10/site-packages/accelerate/__init__.py", line 3, in <module>
    from .accelerator import Accelerator
  File "/opt/conda/lib/python3.10/site-packag

# optim 换成 paged_lion_32bit    lora_rank从8换到16 似乎有可见的波动 loss从0.6到1 而且显存占用明显比paged_adamw_32bit小（14.4G） 才12.9G
# 我试试继续增加lora_rank 到64 
# <font color=red>可以跑 train loss好像波动大 但是eval的margin和acc倒是越来越大</font>

In [9]:
%cd /kaggle/working/MedicalGPT
!git pull --force --all
!python dpo_training.py \
    --model_type chatglm \
    --model_name_or_path THUDM/chatglm2-6b \
    --train_file_dir ./data/reward_yunguan \
    --validation_file_dir ./data/reward_yunguan \
    --learning_rate 1e-5 \
    --warmup_steps 10 \
    --optim  paged_lion_32bit \
    --per_device_train_batch_size 1 \
    --per_device_eval_batch_size 1 \
    --do_train \
    --do_eval \
    --use_peft True \
    --max_train_samples 1000 \
    --max_eval_samples 20 \
    --max_steps 200 \
    --eval_steps 10 \
    --save_steps 40 \
    --save_total_limit 2 \
    --load_best_model_at_end True \
    --max_source_length 256 \
    --max_target_length 128 \
    --output_dir outputs-dpo-yunguan-v1 \
    --target_modules all \
    --lora_rank 64 \
    --lora_alpha 32 \
    --lora_dropout 0.05 \
    --torch_dtype float16 \
    --fp16 True \
    --device_map auto \
    --report_to tensorboard \
    --remove_unused_columns False \
    --gradient_checkpointing True \
    --cache_dir ./cache

/kaggle/working/MedicalGPT
Fetching origin
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), 833 bytes | 277.00 KiB/s, done.
From https://github.com/valkryhx/MedicalGPT
   8138fc4..317eeb7  medGPT_0828 -> origin/medGPT_0828
Updating 8138fc4..317eeb7
Fast-forward
 dpo_training.py | 4 ++++
 1 file changed, 4 insertions(+)
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[2023-08-29 06:32:33,054] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so:

# 查看一下policy_chosen_logps 到底是啥
# 这个reward_accuracies跟gradient_accumulation_steps有关 代码中默认是4 目前gradient_accumulation_steps=4  我在下面调用时显式的写出来
# 所以每次是4对chosen_rewards与rejected_rewards比较 那难怪会在 [0 , 0.25 , 0.5 ,0.75] 之间跳 很离散 而且这样的话train过程中的acc其实没有多大指示性 毕竟只有4对
# 还是要看eval_acc才有用 理解了！
# 参考 https://github.com/valkryhx/MedicalGPT/blob/medGPT_0828/dpo_3.py#L327 和 L334 
 # 求了 reward_acc的mean() 因为reward_acc是类似[1,0,0,1]这种比较结果后得到的True/False 再float()转成1./0.的tensor！参考下面的代码：

In [15]:
import torch
a=torch.tensor([1])
b=torch.tensor([5])
c=(a>b)
print(c)
print(c.float())

tensor([False])
tensor([0.])


In [17]:
%cd /kaggle/working/MedicalGPT
!git pull --force --all
!python dpo_only_for_show_acc.py \
    --model_type chatglm \
    --model_name_or_path THUDM/chatglm2-6b \
    --train_file_dir ./data/reward_yunguan \
    --validation_file_dir ./data/reward_yunguan \
    --learning_rate 1e-5 \
    --warmup_steps 10 \
    --optim  paged_lion_32bit \
    --per_device_train_batch_size 1 \
    --per_device_eval_batch_size 1 \
    --gradient_accumulation_steps 4 \
    --do_train \
    --do_eval \
    --use_peft True \
    --max_train_samples 1000 \
    --max_eval_samples 20 \
    --max_steps 200 \
    --eval_steps 10 \
    --save_steps 40 \
    --save_total_limit 2 \
    --load_best_model_at_end True \
    --max_source_length 256 \
    --max_target_length 128 \
    --output_dir outputs-dpo-yunguan-v1 \
    --target_modules all \
    --lora_rank 64 \
    --lora_alpha 32 \
    --lora_dropout 0.05 \
    --torch_dtype float16 \
    --fp16 True \
    --device_map auto \
    --report_to tensorboard \
    --remove_unused_columns False \
    --gradient_checkpointing True \
    --cache_dir ./cache

Loading checkpoint shards: 100%|██████████████████| 7/7 [01:28<00:00, 12.69s/it]
2023-08-29 08:27:03.930 | ERROR    | __main__:main:574 - id(model)=134526130352192
2023-08-29 08:27:03.930 | ERROR    | __main__:main:575 - id(model_ref)=134526128800048
2023-08-29 08:27:03.934 | INFO     | __main__:main:611 - Peft target_modules: ['dense', 'dense_4h_to_h', 'dense_h_to_4h', 'query_key_value']
2023-08-29 08:27:03.934 | INFO     | __main__:main:620 - ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=Fals

# dpo_for_peftmodel 直接加载一个 adapter
# 这是  qlora的 还是感觉train loss 和eval loss 不怎么收敛 

In [5]:
%cd /kaggle/working/MedicalGPT
!git pull --force --all
!python dpo_for_peftmodel.py \
    --model_type chatglm \
    --model_name_or_path /kaggle/working/chatGLM-6B-QLoRA/output_yungaun_0827_v1/checkpoint-400 \
    --tokenizer_name_or_path THUDM/chatglm2-6b \
    --train_file_dir ./data/reward_yunguan \
    --validation_file_dir ./data/reward_yunguan \
    --learning_rate 1e-5 \
    --warmup_steps 10 \
    --load_in_4bit True \
    --qlora True \
    --optim  paged_lion_32bit \
    --per_device_train_batch_size 1 \
    --per_device_eval_batch_size 1 \
    --gradient_accumulation_steps 4 \
    --do_train \
    --do_eval \
    --use_peft True \
    --max_train_samples 1000 \
    --max_eval_samples 20 \
    --max_steps 200 \
    --eval_steps 10 \
    --save_steps 40 \
    --save_total_limit 2 \
    --load_best_model_at_end True \
    --max_source_length 256 \
    --max_target_length 128 \
    --output_dir outputs-dpo-yunguan-v1 \
    --target_modules all \
    --lora_rank 64 \
    --lora_alpha 32 \
    --lora_dropout 0.05 \
    --torch_dtype float16 \
    --fp16 True \
    --device_map auto \
    --report_to tensorboard \
    --remove_unused_columns False \
    --gradient_checkpointing True \
    --cache_dir ./cache

{'loss': 0.6929, 'learning_rate': 5e-06, 'rewards/chosen': 0.004874515347182751, 'rewards/rejected': 0.004236030858010054, 'rewards/accuracies': 0.625, 'rewards/margins': 0.0006384849548339844, 'logps/rejected': -78.71856689453125, 'logps/chosen': -74.25202941894531, 'logits/rejected': -2.110980272293091, 'logits/chosen': -2.101668357849121, 'epoch': 0.05}
{'loss': 0.6698, 'learning_rate': 6e-06, 'rewards/chosen': 0.08982916176319122, 'rewards/rejected': 0.04108934849500656, 'rewards/accuracies': 0.625, 'rewards/margins': 0.04873981326818466, 'logps/rejected': -96.49650573730469, 'logps/chosen': -87.66525268554688, 'logits/rejected': -1.9265261888504028, 'logits/chosen': -2.0274808406829834, 'epoch': 0.06}
{'loss': 0.663, 'learning_rate': 7e-06, 'rewards/chosen': 0.06729288399219513, 'rewards/rejected': 0.004191780462861061, 'rewards/accuracies': 0.75, 'rewards/margins': 0.06310110539197922, 'logps/rejected': -53.67744064331055, 'logps/chosen': -97.54354858398438, 'logits/rejected': -2

# 试试普通lora 设置--use_ref_model False 让程序自己去create ref model  避免直接copy model构造ref model导致的oom
# 普通lora 当ref_model=None时 lora target modules=[default。。。]居然会混入奇怪的default module 我只好在函数中强制删除
# 普通lora 仍然oom  不能传 load_in_4bit 会导致参与训练的modules为[]
Target modules [] not found in the base model. Please check the target modules and try again.

In [10]:
%cd /kaggle/working/MedicalGPT
!git pull --force --all
!python dpo_for_peftmodel.py \
    --model_type chatglm \
    --model_name_or_path /kaggle/working/chatGLM-6B-QLoRA/output_yungaun_0827_v1/checkpoint-400 \
    --tokenizer_name_or_path THUDM/chatglm2-6b \
    --use_ref_model False \
    --train_file_dir ./data/reward_yunguan \
    --validation_file_dir ./data/reward_yunguan \
    --learning_rate 1e-5 \
    --warmup_steps 10 \
    --load_in_4bit True \
    --qlora False \
    --optim  paged_lion_32bit \
    --per_device_train_batch_size 1 \
    --per_device_eval_batch_size 1 \
    --gradient_accumulation_steps 4 \
    --do_train \
    --do_eval \
    --use_peft True \
    --max_train_samples 1000 \
    --max_eval_samples 20 \
    --max_steps 200 \
    --eval_steps 10 \
    --save_steps 40 \
    --save_total_limit 2 \
    --load_best_model_at_end True \
    --max_source_length 256 \
    --max_target_length 128 \
    --output_dir outputs-dpo-yunguan-lora-v1 \
    --target_modules all \
    --lora_rank 64 \
    --lora_alpha 32 \
    --lora_dropout 0.05 \
    --torch_dtype float16 \
    --fp16 True \
    --device_map auto \
    --report_to tensorboard \
    --remove_unused_columns False \
    --gradient_checkpointing True \
    --cache_dir ./cache

/kaggle/working/MedicalGPT
Fetching origin
Already up to date.
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[2023-08-29 15:56:37,452] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libte